In [1]:
import pandas as pd
import numpy as np

import json

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [ ]:
products = pd.read_pickle("./data/products_pd.pickle")

products['price_dollar'] = np.nan

# Walmart articles

In [11]:
with open('walmart_products.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)

JSONDecodeError: Extra data: line 1 column 281042 (char 281041)

In [10]:
data

{'cells': [{'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import requests\n',
    'from urllib.parse import urljoin\n',
    'from bs4 import BeautifulSoup\n',
    '\n',
    'import json\n',
    '\n',
    'from time import sleep\n',
    '\n',
    'import web_crawler.scraper_walmart as walmart\n',
    '\n',
    '# Our generated code\n',
    'import os\n',
    'import sys\n',
    'nb_dir = os.path.split(os.getcwd())[0]\n',
    'if nb_dir not in sys.path:\n',
    '    sys.path.append(nb_dir)\n',
    '    \n',
    '%load_ext autoreload\n',
    '%autoreload 2']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['step_size = 10\n',
    '\n',
    "#with open('/libs/data.json') as f:\n",
    '#    json_data = json.load(f)\n',
    '\n',
    'products = pd.read_pickle("./data/products_pd.pickle")\n',
    '\n',
    "products['price_do

In [7]:
walmart_products = pd.DataFrame.from_dict(data)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

# Migros articles

# Monoprix articles